In [20]:
import os
token=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT")

In [21]:
from langchain_astradb import AstraDBVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.cassandra import Cassandra

## Configure the embedding model and vector store
embeddings = OllamaEmbeddings(model="llama3")
vectorStore = AstraDBVectorStore(
    collection_name="test",
    embedding=embeddings,
    token=token,
    api_endpoint=api_endpoint
)

In [22]:
from datasets import load_dataset

# Load a sample dsataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [23]:
from langchain.schema import Document

# constructs a set of documnets from data.Documnets can be used as inputs to vector store
docs=[]
for entry in philo_dataset:
    metadata = {"author":entry["author"]}
    if entry["tags"]:
        #Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a lanchain documnet with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [ ]:
docs

In [25]:
# create embeddings by inserting documnets in to the vector store

inserted_ids = vectorStore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [26]:
#Check collection to verify the documents are embedded
print(vectorStore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '5e4b6cbf867044c2b015c48cef7760a7', 'content': 'Whatever fosters the growth of civilization works at the same time against war.', 'metadata': {'author': 'freud', 'knowledge': 'y', 'ethics': 'y'}}, {'_id': '30abe0b2abfc4e809eb658fd24b48f12', 'content': 'Impatience asks for the impossible, wants to reach the goal without the means of getting there. The length of the journey has to be borne with, for every moment is necessary.', 'metadata': {'author': 'hegel', 'ethics': 'y'}}, {'_id': '202ffd58de9349c4bf3e2fc584db329e', 'content': 'A promise made must be a promise kept.', 'metadata': {'author': 'aristotle', 'ethics': 'y'}}, {'_id': '0f89b95463dd4b7a9277ef79717f07e9', 'content': "It is most important to allow the brain the full measure of sleep which is required to restore it; for sleep is to a man's whole nature what winding up is to a clock.", 'metadata': {'author': 'schopenhauer', 'ethics': 'y', 'knowledge': 'y'}}, {'_id': '18fd21d2b6864c1a81ef35527d925eb

In [ ]:
# Retrieve context from the vector atore and pass it to the llm

from langchain.prompts import ChatPromptTemplate
from transformers import pipeline
from langchain.chat_models import ChatOpenAI
from langchain_huggingface import HuggingFacePipeline
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vectorStore.as_retriever(search_kwargs={"k":3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer. 
Context: {context}
Question: {question}
Your Answer: 
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
# model = ChatOpenAI()

# Initialize the Hugging Face pipeline
hf_pipeline = pipeline("text-generation", model="EleutherAI/gpt-neox-20b", max_length=300, max_new_tokens=10)
# EleutherAI/gpt-neo-1.3B
# EleutherAI/gpt-j-6B
# EleutherAI/gpt-neox-20b
# acebook/blenderbot-400M-distill
# facebook/blenderbot-1B-distill
# facebook/blenderbot-3B
# microsoft/DialoGPT-small
# microsoft/DialoGPT-medium
# microsoft/DialoGPT-large
# google/t5-small
# google/t5-base
# google/t5-large
# google/t5-3B
# google/t5-11B
# distilgpt2

# Wrap the pipeline in a LangChain-compatible LLM
model = HuggingFacePipeline(pipeline=hf_pipeline)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("In the given context, what subject are philosophers most concerned with?"))

In [39]:
response = chain.invoke("In the given context, what subject are philosophers most concerned with?, only give the answer")
response

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"Human: \nAnswer the question based only on the supplied context. If you don't know the answer, say you don't know the answer. \nContext: [Document(metadata={'author': 'sartre'}, page_content='your judgement judges you and defines you'), Document(metadata={'author': 'hegel', 'history': 'y', 'knowledge': 'y'}, page_content='We learn from history that we do not learn from history'), Document(metadata={'author': 'nietzsche'}, page_content='Most people are too stupid to act in their own interest')]\nQuestion: In the given context, what subject are philosophers most concerned with?, only give thr answer\nYour Answer: \n\nThe subject of philosophy is the human being."

In [40]:
print(response)

Human: 
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer. 
Context: [Document(metadata={'author': 'sartre'}, page_content='your judgement judges you and defines you'), Document(metadata={'author': 'hegel', 'history': 'y', 'knowledge': 'y'}, page_content='We learn from history that we do not learn from history'), Document(metadata={'author': 'nietzsche'}, page_content='Most people are too stupid to act in their own interest')]
Question: In the given context, what subject are philosophers most concerned with?, only give thr answer
Your Answer: 

The subject of philosophy is the human being.


In [41]:
vectorStore.delete_collection()